# Stack Overflow Data EDA and Text Generation 
- Joel Stremmel
- 01-14-20

**About:**

This notebook builds and trains an encoder decoder RNN for text generation with the Stack Overflow data available through `tff.simulation.datasets` with Federared Averaging by following the Federated Learning for Text Generation example notebook listed in the references section.

**Notes:**

This notebook builds and trains an encoder decoder RNN for text generation.


**Data:** 
- https://www.kaggle.com/stackoverflow/stackoverflow

**License:** 
- https://creativecommons.org/licenses/by-sa/3.0/

**Data and Model References:**
- https://www.tensorflow.org/tutorials/text/nmt_with_attention
- https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow/load_data
- https://github.com/tensorflow/federated/blob/master/docs/tutorials/federated_learning_for_text_generation.ipynb
- https://github.com/tensorflow/federated/
- https://www.tensorflow.org/tutorials/text/text_generation

**Environment Setup References:**
- https://www.tensorflow.org/install/gpu
- https://gist.github.com/matheustguimaraes/43e0b65aa534db4df2918f835b9b361d
- https://www.tensorflow.org/install/source#tested_build_configurations
- https://anbasile.github.io/programming/2017/06/25/jupyter-venv/

### Setup

In [1]:
# !pip install --upgrade pip
# !pip install --upgrade tensorflow-federated
# !pip uninstall tensorflow -y
# !pip install --upgrade tensorflow-gpu==2.0
# !pip install --upgrade nltk
# !pip install matplotlib
# !pip install nest_asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import collections
import functools
import os
import six
import time
import string

import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [4]:
import tensorflow as tf
print('Built with Cuda: {}'.format(tf.test.is_built_with_cuda()))
print('Build with GPU support: {}'.format(tf.test.is_built_with_gpu_support()))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Built with Cuda: True
Build with GPU support: True
Num GPUs Available:  1


In [5]:
import tensorflow_federated as tff

In [6]:
tf.compat.v1.enable_v2_behavior()

In [7]:
np.random.seed(0)

### Set Tensorflow to Use GPU

In [8]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
physical_devices = tf.config.experimental.list_physical_devices(device_type=None)
tf.config.experimental.set_memory_growth(physical_devices[-1], enable=True)
for device in physical_devices:
    print(device)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')
PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### Test TFF

In [10]:
tff.federated_computation(lambda: 'Hello, World!')()

'Hello, World!'

### Load Stack Overflow Data

In [23]:
train_data, val_data, test_data = tff.simulation.datasets.stackoverflow.load_data(cache_dir='~/data')
# print(train_data.output_shapes)

/home/joel_stremmel/anaconda3/envs/tff/lib/python3.7/site-packages/tensorflow_federated/python/simulation/hdf5_client_data.py:69: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  collections.OrderedDict((name, ds.value) for name, ds in sorted(


### Count Number of Clients

In [25]:
print('{} train clients.'.format(len(train_data.client_ids)))
print('{} val clients.'.format(len(val_data.client_ids)))
print('{} test clients.'.format(len(test_data.client_ids)))

342477 train clients.
38758 val clients.
204088 test clients.


### Get Sample Clients for Training, Validation, and Testing

In [26]:
NUM_TRAIN_CLIENTS = 50
NUM_VAL_CLIENTS = 10
NUM_TEST_CLIENTS = 10

In [27]:
def get_sample_clients(dataset, num_clients):
    return np.array(dataset.client_ids)[np.random.choice(len(dataset.client_ids),
                                                         size=num_clients,
                                                         replace=False)]

In [28]:
train_clients = get_sample_clients(train_data, num_clients=NUM_TRAIN_CLIENTS)
val_clients = get_sample_clients(val_data, num_clients=NUM_VAL_CLIENTS)
test_clients = get_sample_clients(test_data, num_clients=NUM_TEST_CLIENTS)

### Set Vocabulary
- Currently using the fixed vocabularly of ASCII chars that occur in the works of Shakespeare and Dickens
- **Is there a good way to get the distinct characters from a TF dataset?**

In [ ]:
vocab = list('dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r')
vocab_size = len(vocab)

### Creating a Mapping from Unique Characters to Indices

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

### Build an Encode Decoder RNN
Text generation requires a batch_size=1 model.

In [ ]:
def build_model(batch_size, vocab_size, embedding_dim=256, rnn_units=512):
    """
    Build model with architecture from: https://www.tensorflow.org/tutorials/text/text_generation.
    """

    model = tf.keras.Sequential([
        
        tf.keras.layers.Embedding(vocab_size,
                                  embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True, 
                            recurrent_initializer='glorot_uniform'),
                                 
#         tf.keras.layers.LSTM(rnn_units,
#                              return_sequences=True,
#                              stateful=True,
#                              recurrent_initializer='glorot_uniform'),
                                 
        tf.keras.layers.Dense(vocab_size)])
                 
    return model

In [ ]:
class Encoder(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state

    def initialize_hidden_state(self):
        
    return tf.zeros((self.batch_size, self.enc_units))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    
    def __init__(self, units):
        
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
    
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
def generate_text(model, start_string):
    """
    Generate text by sampling from the model output distribution
    as in From https://www.tensorflow.org/tutorials/sequences/text_generation.
    """

    num_generate = 200
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
# keras_model_batch1 = load_pretrained_model(batch_size=1)
keras_model_batch1 = build_model(batch_size=1, vocab_size=vocab_size)
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

### Preprocess Federated Stack Overflow
- Using a namedtuple with keys x and y as the output type of the dataset keeps both TFF and Keras happy.
- Construct a lookup table to map string chars to indexes, using the vocab loaded above.
- Write functions for:
    - ID lookup
    - Splitting inputs and targets
    - Applying preprocessing steps to dataset
    - Taking clients and client records and applying preprocessing

In [ ]:
SEQ_LENGTH = 100
BATCH_SIZE = 16
BUFFER_SIZE = 5000

In [ ]:
BatchType = collections.namedtuple('BatchType', ['x', 'y'])

In [ ]:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab,
        values=tf.constant(list(range(len(vocab))),
        dtype=tf.int64)),
    default_value=0)

In [ ]:
def to_ids(x):
    
    s = tf.reshape(x['tokens'], shape=[1])
    chars = tf.strings.bytes_split(s).values
    ids = table.lookup(chars)
    
    return ids

In [ ]:
def split_input_target(chunk):
    
    input_text = tf.map_fn(lambda x: x[:-1], chunk)
    target_text = tf.map_fn(lambda x: x[1:], chunk)
    
    return BatchType(input_text, target_text)

In [ ]:
def preprocess(dataset):
    
    return (
        # Map ASCII chars to int64 indexes using the vocab
        dataset.map(to_ids)
        # Split into individual chars
        .unbatch()
        # Form example sequences of SEQ_LENGTH +1
        .batch(SEQ_LENGTH + 1, drop_remainder=True)
        # Shuffle and form minibatches
        .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
        # And finally split into (input, target) tuples,
        # each of length SEQ_LENGTH.
        .map(split_input_target))

In [ ]:
def preprocess_data_for_client(client, source=train_data):
    
    return preprocess(source.create_tf_dataset_for_client(client))

In [ ]:
train_datasets = [preprocess_data_for_client(client, train_data) for client in train_clients]
print(tf.data.experimental.get_structure(train_datasets[0]))

### Compile and Test on Preprocessed Data

In [ ]:
class FlattenedCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

    def __init__(self, name='accuracy', dtype=None):
        super(FlattenedCategoricalAccuracy, self).__init__(name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_true = tf.reshape(y_true, [-1, 1])
        y_pred = tf.reshape(y_pred, [-1, len(vocab), 1])
        
        return super(FlattenedCategoricalAccuracy, self).update_state(y_true, y_pred, sample_weight)

In [ ]:
def loss_function(real, pred, loss_objective):

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_objective(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
def compile(keras_model, loss_objective):
    
    keras_model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.005), # updated from SGD; TO DO: experiment with a scheduler
        loss=loss_objective,
        metrics=[FlattenedCategoricalAccuracy()]
    )
    
    return keras_model

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
loss_objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

### Load and Compile Model

In [ ]:
keras_model = build_model(batch_size=BATCH_SIZE, vocab_size=vocab_size)
compile(keras_model)

### Improve Model with Federated Averaging
- Clone the keras_model inside `create_tff_model()`, which TFF will call to produce a new copy of the model inside the graph that it will serialize.
- TFF uses a `dummy_batch` so it knows the types and shapes that your model expects.
- Build and serialize the Tensorflow graph with `build_federated_averaging_process`.

In [ ]:
def create_tff_model():
    
    x = tf.constant(np.random.randint(1, len(vocab), size=[BATCH_SIZE, SEQ_LENGTH]))
    dummy_batch = collections.OrderedDict([('x', x), ('y', x)]) 
    keras_model_clone = compile(tf.keras.models.clone_model(keras_model))
    
    return tff.learning.from_compiled_keras_model(keras_model_clone, dummy_batch=dummy_batch)

In [ ]:
fed_avg = tff.learning.build_federated_averaging_process(model_fn=create_tff_model)

### Run One Round of Federated Averaging

In [ ]:
state = fed_avg.initialize()
state, metrics = fed_avg.next(state, [train_datasets[0].take(1)])
print(metrics)

### Build and Preprocess the Validation and Test Datasets
Concatenate the validation and test datasets for evaluation with Keras.

In [ ]:
val_dataset = functools.reduce(lambda d1, d2: d1.concatenate(d2), 
                               [preprocess_data_for_client(client, val_data) for client in val_clients])

test_dataset = functools.reduce(lambda d1, d2: d1.concatenate(d2), 
                                [preprocess_data_for_client(client, test_data) for client in test_clients])

### Train with Federated Averaging

In [ ]:
# NOTE: If the statement below fails, it means that you are
# using an older version of TFF without the high-performance
# executor stack. Call `tff.framework.set_default_executor()`
# instead to use the default reference runtime.
if six.PY3:
    tff.framework.set_default_executor(tff.framework.create_local_executor())

In [ ]:
NUM_ROUNDS = 20

# The state of the FL server, containing the model and optimization state.
state = fed_avg.initialize()

state = tff.learning.state_with_new_model_weights(
    state,
    trainable_weights=[v.numpy() for v in keras_model.trainable_weights],
    non_trainable_weights=[v.numpy() for v in keras_model.non_trainable_weights]
)

def keras_evaluate(state, round_num):
    tff.learning.assign_weights_to_keras_model(keras_model, state.model)
    print('Evaluating before training round', round_num)
    keras_model.evaluate(val_dataset, steps=2)

for round_num in range(NUM_ROUNDS):
    keras_evaluate(state, round_num)
    state, metrics = fed_avg.next(state, train_datasets)
    print('Training metrics: ', metrics)

keras_evaluate(state, NUM_ROUNDS + 1)

### Generate Text
Text generation requires batch_size=1.

In [ ]:
keras_model_batch1.set_weights([v.numpy() for v in keras_model.weights])
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask?'))

**Suggested extensions from tutorial:**

- Write a more realistic training loop where you sample clients to train on randomly.
- Use ".repeat(NUM_EPOCHS)" on the client datasets to try multiple epochs of local training (e.g., as in McMahan et. al.). See also Federated Learning for Image Classification which does this.
- Change the compile() command to experiment with using different optimization algorithms on the client.
- Try the server_optimizer argument to build_federated_averaging_process to try different algorithms for applying the model updates on the server.
- Try the client_weight_fn argument to to build_federated_averaging_process to try different weightings of the clients. The default weights client updates by the number of examples on the client, but you can do e.g. client_weight_fn=lambda _: tf.constant(1.0).